# Fix existing faulty AMBuildPlates
2022-02-02

Drop ones with bad ID

In [ ]:
import sys
import os
import io
import pprint
# import lxml
import lxml.etree as ET
import xml.dom.minidom
import importlib
import glob
import json
import string
import datetime
import pandas
import getpass
import matplotlib.pyplot as plt
import requests
from PIL import Image, ImageDraw, ImageFont

In [ ]:
import pyxb
pyxb.RequireValidWhenGenerating(False);

# preliminaries

In [ ]:
import myconfig

In [ ]:
importlib.reload(myconfig)
from myconfig import *

In [ ]:
AMBENCH=AMBENCH_URL.split("/")[2].split(".")[0]
EXCEL_FILE = SAMPLES_EXCEL_FILE
try : USER
except: USER = input('username: ')
try: PASS
except : PASS = getpass.getpass('enter password ')
AUTH=(USER, PASS)

In [ ]:
sys.path.insert(0, pyUTILS_path)
import ambench.cdcs_utils
from ambench.mapping import new_mapper

In [ ]:
importlib.reload(ambench.cdcs_utils)
from ambench.cdcs_utils import *

# create AMBench2022 instance

In [ ]:
AMBENCH_URL='https://test-ambench2022.nist.gov/'
ambench2022=AMBench2022(TEMPLATE,AMBENCH_URL,auth=AUTH)
# create validator from in-memory XSDs
CDCS_VALIDATOR = ambench2022.create_schema_validator(); 

In [ ]:
xsd_filename=f'{XSD}AMDocs.xsd'
VALIDATOR=xmlschema.XMLSchema(xsd_filename,build=False)
VALIDATOR.build()
VALIDATOR.validity 

# Retrieve the docs

In [ ]:
DOC_TYPE='AMSpecimen'

outfolder=f"{ROOT}/xml/{DOC_TYPE}/"
os.makedirs(outfolder,exist_ok=True)

In [ ]:
docs=ambench2022.query_docs_by_type(DOC_TYPE)

## change element tag name
AMSpecimen -> AMBSpecimen

In [ ]:
import xml.etree.ElementTree as ET
import io

In [ ]:
files=[]
for t in docs.itertuples():
    xml_content=t.xml_content
    tree=ET.parse(io.StringIO(xml_content))
    for elem in tree.findall("AMSpecimen"):
        elem.tag = "AMBSpecimen"
    file=f'{outfolder}{t.title}'
    files.append((t.id,file))
    tree.write(file)

In [ ]:
for _id,xmlfile in files:
#     print(xmlfile)
    v=CDCS_VALIDATOR.is_valid(xmlfile)
    if not(v):
        try:
            VALIDATOR.validate(xmlfile)
        except Exception as e:
            print(xmlfile,"\n",e,"\n=====\n")
            raise e
print('DONE')

## load to CDCS

In [ ]:
uploaded={}
errors={}
for _id,f in files:
    # TODO check whether pid exists and if so update rather than upload??
    fn=os.path.basename(f)
    print('update existing:', f)
    response=ambench2022.update_data(f)

    if response.ok:
        uploaded[fn]=response.json()    
    else:
        errors[fn]=response.json()

# remove duplicates were somehow upploaded iso updating existing
Delete duplicates

In [ ]:
MQ={"AMDoc.AMBSpecimen":{"$exists":True}}
df=ambench2022.mongo_query(MQ)

In [ ]:
groups=df.groupby(['title'])

In [ ]:
bad=[]
nbad=0
ngood=0
for t,g in groups:
    if len(g)>1:
        gbad=g[g['creation_date']>='2022-02-01']
        ggood=g[g['creation_date']<'2022-02-01']
        bad.append(gbad)
        nbad+=len(gbad)
        ngood+=len(ggood)
nbad,ngood

In [ ]:
for b in bad:
    for row in b.itertuples():
        print("delete_record(",row.id,row.title,")")
        ambench2022.delete_record(row)

In [ ]:
df